# Recommendation API

## A Flask aplication to recommend items to a specific user based on precalculated file

In this notebook we build a flask API Endpoint that loads a csv file offline. The API has only one endpoint: /ratings/top. By calling (POST) this endpoint with a userId en optionally a count in the body, the top recommended items together with the prediction scores are returned.

To avoid you should use Spark, this notebook is entirely based on Python.

We need to run the entire application in one paragraph, because it needs to be constantly up and running.

In [34]:
import pandas as pd 
predictions = pd.read_csv("C:/Users/ckadic/Desktop/RecommendationTools/GroupAssignmentData/Data-20190228/predictions.csv").dropna()
predictions.head()

,Unnamed: 0,userId,movieId,rating,timestamp,prediction
0,0,126,471,5.0,833287141,3.850055
1,1,86,471,4.0,848161161,4.147853
2,2,491,471,3.0,940797129,4.703628
3,3,514,471,4.0,853893788,4.066665
4,4,102,471,5.0,958248997,4.230568


In [35]:
movies = pd.read_csv("C:/Users/ckadic/Desktop/RecommendationTools/GroupAssignmentData/Data-20190228/movies.csv").dropna()
movies.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [40]:
## Import packages
from flask import Flask, request, jsonify
import pandas as pd 

## Initialize app
app = Flask(__name__)

## Read in predictions.csv and movies.csv using Pandas
predictions = pd.read_csv("C:/Users/ckadic/Desktop/RecommendationTools/GroupAssignmentData/Data-20190228/predictions.csv").dropna()

movies = pd.read_csv("C:/Users/ckadic/Desktop/RecommendationTools/GroupAssignmentData/Data-20190228/movies.csv").dropna()

## Combine both files to allow to return complete information about the movie
predictions = predictions.merge(movies, on="movieId", how="left").sort_values(['userId', 'prediction'], ascending=[True, False])

## How this is done in Spark (information)
#movies = spark.read.option("header", "true").csv("movies.csv")\
#                    .select("movieId", "title")\
#                    .repartition("movieId")
            
#predictions = predictions.join(movies, "movieId", "left")\
#                    .orderBy(col("userId"), col("prediction").desc())\
#                    .cache()

## The application definiton
### Endpoint - one route /ratings/top - one HTTP verb = POST
@app.route("/ratings/top", methods=["POST"])
def top_ratings():
    ## read the body of the API call
    content = request.get_json()
    
    ## Interpretation of body
    if "userId" in content and type(content["userId"]) == int:
        userId = content["userId"]
    else:
        return "'userId' is required and should be an Integer."
        sys.exit("'userId' is required and should be an Integer.")
        
    if "count" in content and type(content["count"]) == int:
        count = content["count"]
    else:
        count = 5
    
    # filter predictions for the given userId
    predict = predictions[predictions.userId == userId].head(count)
    
    # select movieId, title and prediction and transform to list
    top_ratings = list(predict[["movieId", "title", "prediction"]].T.to_dict().values())
    
    # Return the result to the API
    return jsonify(top_ratings)

### Put endpoint online
if __name__ == 'REST':
    app.run(host='localhost', port=6000)